In [ ]:
from math import exp
from random import random
import numpy as np

In [ ]:
# Initialize the network
def initializeNetwork(nInputs, nHidden, nOutputs):
	network = []
	hiddenLayer = [{'weights':[random() for i in range(nInputs + 1)]} for i in range(nHidden)]
	network.append(hiddenLayer)
	outputLayer = [{'weights':[random() for i in range(nHidden + 1)]} for i in range(nOutputs)]
	network.append(outputLayer)
	return network

# Calculate the logit
def neuronLogit(weights, input):
	logit = weights[-1]
	for i in range(len(weights)-1):
		logit += weights[i] * input[i]
	return logit

# Use sigmoid as squash function
def sigmoid(z):
	return 1 / (1 + exp(-z))

# Do the Feed forward propagation
def forwardPropagation(network, example):
	inputs = example
	for layer in network:
		newInputs = []
		for neuron in layer:
			activate = neuronLogit(neuron['weights'], inputs)
			neuron['output'] = sigmoid(activate)
			newInputs.append(neuron['output'])
		inputs = newInputs
	return inputs

# Calculate the sigmoid derivative
def sigmoidDerivative(output):
	return output * (1 - output)

# Do the Backpropagation
def backPropagation(network, target):
	for i in reversed(range(len(network))):
		layer = network[i]
		error = []
		if i != len(network)-1:
			for j in range(len(layer)):
				err = 0.0
				for neuron in network[i + 1]:
					err += (neuron['weights'][j] * neuron['delta'])
				error.append(err)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				error.append(neuron['output'] - target)
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = error[j] * sigmoidDerivative(neuron['output'])

# Update the weights from Backpropagation
def updateWeights(network, example, eta):
	for i in range(len(network)):
		inputs = example[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= eta * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= eta * neuron['delta']

# Train the network
def trainNetwork(network, trainData, eta, nEpoch, target):
	for epoch in range(nEpoch):
		for i in range(len(trainData)):
			forwardPropagation(network, trainData[i])
			backPropagation(network, target[i][0])
			updateWeights(network, trainData[i], eta)


def readFile(path):
    with open(path) as f:
        dataset = [list(map(float, (line.split()))) for line in f.readlines()]
        dataset, target = [line[:2] for line in dataset], [[line[-1]] for line in dataset]
    return dataset, target




# initialize network with 2,4,1
trainX, trainY = readFile('data2.txt')
nInputs = len(trainX[0])
nOutputs = 1
network = initializeNetwork(nInputs, 4, nOutputs)

# train the network
trainNetwork(network, trainX, 0.005, 10000, trainY)
#for layer in network:
	#print(layer)

In [ ]:
# Make a prediction with a network
def predict(network, row):
	outputs = forwardPropagation(network, row)
	return outputs


def validation(testdata,target, thresh=0.5, verbose=0):
    result = []
    for row in testdata:
        result.append(predict(network, row))
    #print(prediction)
    SSE = 0
    for i in range(len(target)):
        difference = target[i][0] - result[i][0]
        #print(difference)
        SSE += (difference*difference)
       
    prediction =  [1 if val[0]>=thresh else 0 for val in result]
    accuracy = sum([pred==actual[0] for pred, actual in zip(prediction, target)])/len(result)
    if verbose:
        print("Accuracy:", accuracy)
        print(result)
        print(prediction)
        print(target)
    return SSE,prediction

# Learned Weights
w_h = [layer['weights'] for layer in network[0]]
w_O = [layer['weights'] for layer in network[1]]

testX, testY = readFile('valid2.txt')  
SSE,prediction = validation(testX,testY, verbose=True)
#print(w_h, w_O)
#print(SSE)

In [ ]:
network = initializeNetwork(nInputs, 4, nOutputs)
trainNetwork(network, trainX, 1, 100, trainY)
SSE,prediction = validation(testX, testY, verbose=True)
#print(w_h, w_O)
#print(SSE)